In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [3]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [4]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [5]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [6]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8-0.2-0.11
    resonator_length_2.append(a)
print(resonator_length_2)


[7.343254683870284, 7.114788339841421, 6.89834654020913]


# Flipchip

In [7]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+80}um",
                                              pad_width="110um",
                                              pad_height="504um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [8]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [9]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [10]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [11]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [12]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [13]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [14]:
hfss.start()

INFO 09:57AM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:57AM [load_ansys_project]: 	Opened Ansys App
INFO 09:57AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 09:57AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project20
INFO 09:57AM [connect_design]: No active design found (or error getting active design).
INFO 09:57AM [connect]: 	 Connected to project "Project20". No design detected


True

In [15]:
hfss.activate_ansys_design("flip_V9", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

09:57AM 25s WARNING [activate_ansys_design]: The design_name=flip_V9 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 09:57AM [connect_design]: 	Opened active design
	Design:    flip_V9 [Solution type: Eigenmode]
WARNING 09:57AM [connect_setup]: 	No design setup detected.
WARNING 09:57AM [connect_setup]: 	Creating eigenmode default setup.
INFO 09:57AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [15]:
hfss.close()

True

# LOM Analysis

In [16]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [17]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.63',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.63',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [19]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 10:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpjqy_abyr.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5630000000, Maxwell, 1, False
INFO 10:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpffahhrx7.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5630000000, Maxwell, 1, False
INFO 10:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpjrhsyzcr.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5630000000, Maxwell, 2, False
INFO 10:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpz5mzamx3.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5630000000, Maxwell, 3, False
INFO 10:06AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpx3b40huf.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5630000000, Maxwell, 4, False
INFO 10:06

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,386.63960,-74.54805,-54.05504,-198.46420
pad_bot_Q0,-74.54805,107.80450,-31.52057,-0.63829
pad_top_Q0,-54.05504,-31.52057,107.40110,-20.81143
quad_coupler_c0,-198.46420,-0.63829,-20.81143,221.02330


In [20]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.69
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.405897 [GHz]
EC 272.689908 [MHz]
EJ 14.854145 [GHz]
alpha -310.930205 [MHz]
dispersion 5.014052 [KHz]
Lq 10.995564 [nH]
Cq 71.033902 [fF]
T1 25.601869 [us]

**Coupling Properties**

tCqbus1 -10.109247 [fF]
gbus1_in_MHz -138.484564 [MHz]
χ_bus1 -5.906384 [MHz]
1/T1bus1 6216.536198 [Hz]
T1bus1 25.601869 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.736231,309.312222,14.854145,-356.539203,17.181976,[-18.345143898814936],[-0.1936272282959192],0.193627,18.345144
2,5.656377,300.219077,14.854145,-345.134054,12.929802,[-208.47805015655524],[-21.25424114550477],21.254241,208.478050
3,5.570522,290.614186,14.854145,-333.145034,9.43657,[-190.02392848742176],[-14.959405139834965],14.959405,190.023928
4,5.574853,291.094505,14.854145,-333.743166,9.589966,[-172.04720790055435],[-12.363438764523032],12.363439,172.047208
5,5.527215,285.8364,14.854145,-327.203425,8.020139,[-155.60412412780684],[-9.254589837353846],9.254590,155.604124
6,5.478201,280.482912,14.854145,-320.5633,6.650755,[-145.19980006153983],[-7.372962234622049],7.372962,145.199800
7,5.450369,277.468473,14.854145,-316.832473,5.970881,[-142.38703457377378],[-6.74788935368737],6.747889,142.387035
8,5.432939,275.589983,14.854145,-314.510503,5.577766,[-139.5427144727015],[-6.285520083728778],6.285520,139.542714
9,5.414999,273.664139,14.854145,-312.132344,5.19776,[-139.2617963604733],[-6.067671354367348],6.067671,139.261796
10,5.405897,272.689908,14.854145,-310.930205,5.014052,[-138.48456433298668],[-5.906383863996053],5.906384,138.484564


In [21]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
